### <div align="center">***DATA CLEANING AND EDA***</div>

### ***Import libraries and modules***

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from pathlib import Path

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from datetime import datetime
from geopy.distance import geodesic   
from haversine import haversine, Unit

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# import warnings
# warnings.filterwarnings('ignore')

### ***Data Loading and exploration***

In [2]:
# Get data path
BASE_DIR = Path.cwd().parent 
data_path = BASE_DIR / "Data" / "Raw" / "amazon_delivery.csv"

# Load dataset
data = pd.read_csv(data_path)
print("Data loaded successfully")

Data loaded successfully


In [3]:
# View top 5 rows of the dataset 
data.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [4]:
# View bottom 5 rows of the dataset 
data.tail()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
43734,jlxf819993117,30,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,11:35:00,11:45:00,Windy,High,motorcycle,Metropolitian,160,Home
43735,aevx342135787,21,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,19:55:00,20:10:00,Windy,Jam,motorcycle,Metropolitian,180,Jewelry
43736,xnek760674819,30,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,23:50:00,00:05:00,Cloudy,Low,scooter,Metropolitian,80,Home
43737,cynl434665991,20,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,13:35:00,13:40:00,Cloudy,High,motorcycle,Metropolitian,130,Kitchen
43738,nsyz997960170,23,4.9,23.351058,85.325731,23.431058,85.405731,2022-03-02,17:10:00,17:15:00,Fog,Medium,scooter,Metropolitian,180,Cosmetics


In [ ]:
# Dataset information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43739 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43739 non-null  object 
 1   Agent_Age        43739 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43739 non-null  float64
 4   Store_Longitude  43739 non-null  float64
 5   Drop_Latitude    43739 non-null  float64
 6   Drop_Longitude   43739 non-null  float64
 7   Order_Date       43739 non-null  object 
 8   Order_Time       43739 non-null  object 
 9   Pickup_Time      43739 non-null  object 
 10  Weather          43648 non-null  object 
 11  Traffic          43739 non-null  object 
 12  Vehicle          43739 non-null  object 
 13  Area             43739 non-null  object 
 14  Delivery_Time    43739 non-null  int64  
 15  Category         43739 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.3+ MB


In [6]:
# No. of rows & columns in the dataset
print(f'No. of rows in the dataset: {data.shape[0]}')
print(f'No. of columns in the dataset: {data.shape[1]}')

No. of rows in the dataset: 43739
No. of columns in the dataset: 16


In [8]:
# No. of duplicates in the dataset
print(f'No. of duplicate rows found in the dataset: {data.duplicated().sum()}')

No. of duplicate rows found in the dataset: 0


In [48]:
# No. of missing values in the dataset
def missing_values(df):

    total_rows = df.shape[0]
    missing_rows = df.isnull().sum().sum()
    non_missing_rows = total_rows - missing_rows

    print(f'Total no. of rows in dataset: {total_rows}')
    print(f'Total no. of missing rows in dataset: {missing_rows}')
    print(f'Total no. of non-missing rows in dataset: {non_missing_rows}\n')

    missing_count = df.isnull().sum()
    missing_percent = 100 * df.isnull().sum() / len(df)
    
    summary = pd.DataFrame({
        'Missing Values': missing_count,
        'Percentage': missing_percent.round(2)
    })
    
    print('Missing Data summary:')
    summary = summary[summary['Missing Values'] > 0]
    return summary.sort_values(by='Percentage', ascending=False)

summary = missing_values(data)
print(summary)

Total no. of rows in dataset: 43739
Total no. of missing rows in dataset: 145
Total no. of non-missing rows in dataset: 43594

Missing Data summary:
              Missing Values  Percentage
Weather                   91        0.21
Agent_Rating              54        0.12


In [ ]:
# Descriptive statistics
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Agent_Age,43739.0,29.567137,5.815155,15.000000,25.000000,30.000000,35.000000,50.000000
Agent_Rating,43685.0,4.633780,0.334716,1.000000,4.500000,4.700000,4.900000,6.000000
Store_Latitude,43739.0,17.210960,7.764225,-30.902872,12.933298,18.551440,22.732225,30.914057
Store_Longitude,43739.0,70.661177,21.475005,-88.366217,73.170283,75.898497,78.045359,88.433452
Drop_Latitude,43739.0,17.459031,7.342950,0.010000,12.985996,18.633626,22.785049,31.054057
Drop_Longitude,43739.0,70.821842,21.153148,0.010000,73.280000,76.002574,78.104095,88.563452
Delivery_Time,43739.0,124.905645,51.915451,10.000000,90.000000,125.000000,160.000000,270.000000


In [50]:
data.describe(include='object').T

,count,unique,top,freq
Order_ID,43739,43739,nsyz997960170,1
Order_Date,43739,44,2022-03-15,1141
Order_Time,43739,177,21:55:00,460
Pickup_Time,43739,193,21:30:00,481
Weather,43648,6,Fog,7440
Traffic,43739,5,Low,14999
Vehicle,43739,4,motorcycle,25527
Area,43739,4,Metropolitian,32698
Category,43739,16,Electronics,2849


#### <div align="center">***Dataset Understanding***</div>
---

- ***The dataset contains detailed information about e-commerce deliveries*** (order details, delivery agent demographics, geographic coordinates, timestamps, & contextual factors such as traffic, weather, and area type).
- The ***target variable*** is ***Delivery_Time*** (in hours), which represents the actual time taken to complete a delivery.
- The dataset consists of ***43739 rows*** & ***16 columns***. 
- It is a ***fairly clean dataset*** with ***no duplicate values*** and ***only 0.33% missing values*** overall (Weather = 91, Agent_Rating = 54).
- ***Some columns need data type conversion*** (dates and times should be converted to datetime).

#### <div align="center">***Variable Understanding***</div>
---
***Numerical variables: 7***

- ***Agent_Age:*** 
  - Age of the delivery agent.
  - Values range from 15-50, with most agents in their 25s-35s. 
  - Distribution appears normal with no extreme outliers.
  
- ***Agent_Rating:*** 
  - Rating of the delivery agent.
  - Values range from 1–6 (though expected to be on a 1–5 scale). 
  - Distribution skews towards higher ratings, suggesting most agents are rated positively.
  - There are 54 missing values and the value 6 may indicate a data entry anomaly which needs cleaning or capping. 
  
- ***Store_Latitude/Longitude:*** 
  - Geographic location of the store.
  - Appears consistent with no obvious invalid coordinates.
  
- ***Drop_Latitude/Longitude:*** 
  - Geographic location of the delivery address.
  - Appears consistent with no obvious invalid coordinates.
  - Useful for calculating distances.
  
- ***Delivery_Time:*** **Target variable** 
  - Represents the actual time taken for delivery (in hours).
  - Ranges from 10-270 hours, ost deliveries are completed within 4–11 days, though a few extreme long deliveries exist (up to ~11 days).
  - Distribution shows right skew → a few long deliveries (possible outliers).
---
***Categorical vaariables: 9***

- ***Order_ID:*** 
  - Unique identifier for each order.
  - No duplicates observed. Can be dropped as it is purely an identifier column.
  
- ***Order_Date/Order_Time:*** 
  - Date and time when the order was placed. Both are of object data types.
  - Convert to datetime format & extract time-based features (day of week, hour, etc).
  
- ***Pickup_Time:*** 
  - Time when the delivery agent picked up the order.
  - Object data type, convert to datetime format & use it to calculate order-to-pickup delay.

- ***Weather:*** 
  - Weather conditions during delivery.
  - Has 6 unique categories of weather with Fog being the most frequent weather type.
  - Also has 91 missing values that needs handling.
  
- ***Traffic:*** 
  - Traffic conditions during delivery.
  - Has 5 differents categories of traffic with traffic being Low most of the times.
 
- ***Vehicle:*** 
  - Mode of transportation used for delivery.
  - Has 4 differents categories of vehicle with most used transportation being motorcycle.
  
- ***Area:*** 
  - Type of delivery area (Urban/Metropolitan).
  - Has 4 differents categories of Area with most orders being delivered in a Metropolitan area.

- ***Category:*** 
  - Category of the product being delivered.
  - Has 16 different types of products being delivered with Electronics being the most delivered product.

### ***Data Cleaning***